## 0. 참조
* https://deepnote.com/@marco-cerrato/EDA-for-NYC-Taxi-Trip-Duration-i2bcXB4ZQf2KOHfAY61n7Q
* https://www.kaggle.com/munmun2004/nyc-taxi
* https://www.kaggle.com/gguddu/new-york-taxi-duration

## 1. 라이브러리 import 및 데이터 확인하기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy.stats import skew, norm
from haversine import haversine, Unit
from scipy.stats import skew, kurtosis
sns.set() 
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/nyc-taxi-trip-duration/train.zip')
test = pd.read_csv('../input/nyc-taxi-trip-duration/test.zip')
sample_submission = pd.read_csv('../input/nyc-taxi-trip-duration/sample_submission.zip')

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
train.info() # 데이터 셋에 존재하는 칼럼명과 갈럼별 결축치, 데이터타입을 확인.

1. 변수 | 설명
------- | -------
**id** | 각 여행에 대한 식별자
**vendor_id** | 여행 기록과 관련된 제공자를 나타내는 코드 
**pickup_datetime** | 미터가 작동 된 날짜 및 시간  
**dropoff_datetime** | 미터가 작동되지 않는 날짜 및 시간  
**passenger_count** | 차량의 승객 수 (운전자 입력 값) 
**pickup_longitude** | 미터가 사용 된 경도
**pickup_latitude** | 미터가 사용 된 위도 
**dropoff_longitude** | 미터가 사용 안 된 경도
**dropoff_latitude** | 미터가 사용 안 된 위도
**store_and_fwd_flag** | 이 플래그는 차량이 서버에 연결되어 있지 않기 때문에 공급 업체에 보내기 전에 여행 레코드가 차량 메모리에 보유되었는지 여부를 나타냅니다 (Y = 저장 및 전달, N = 저장 및 전달 여행이 아님).
**trip_duration** | 여행 시간 (초)

>1. 데이터셋는 1458644개의 rows와 11개의 columns을 갖는다.
>2. 데이터 타입에는 4개의 실수, 3개의 정수, 4개의 오브젝트로 나뉜다.
>3. null값은 없다.
>4. pickup_datetime, dropoff_datetime은 object이다. 이것을 datetime 타입으로 바꿔주자.
>5. 10개의 독립변수와 1개의 종속변수로 이루어져 있다.

In [ ]:
train.isnull().sum() # 결측치 확인하기

In [ ]:
train.nunique() # 유일한 값(고유값) 개수 찾기

## 2. 숫자, 비숫자 변수들의 통계 요약해보기

In [ ]:
# 숫자 변수들의 통계 요약
train.describe()

위 그래프를 보고 알 수 있는 점
* vendor_id는 1,2로만 되어있다.
* passenger_count는 보통 1~2명의 승객이고, 최대 9명의 승객도 탔다.
* trip_duration의 max부분에서 말도 안되는 시간이 존재한다.

In [ ]:
# 숫자가 아닌 변수들의 통계 요약
train.describe (include=object)

위 그래프를 보고 알 수 있는 점
* id에서 freq(frequency)가 1임을 봤을 때, unique한 값이다.

## 3. 옳은 데이터 타입으로 바꾸기

pickup_datetime과 dropoff_dateime 을 Datetime 데이터타입으로 바꿀 것이다. 이를 통해 새로운 feature를 만들 수 있음.

In [ ]:
# object -> datetime
train['pickup_datetime']= pd.to_datetime(train['pickup_datetime'])
train['dropoff_datetime']= pd.to_datetime(train['dropoff_datetime'])

In [ ]:
# 변환 확인
print(train['pickup_datetime'].dtypes)
print(train['dropoff_datetime'].dtypes)

In [ ]:
# Creating features based on month

train['pickup_by_month'] = train['pickup_datetime'].dt.month
train['dropoff_by_month'] = train['dropoff_datetime'].dt.month

# Creating features based on weekday

train['pickup_by_weekday'] = train['pickup_datetime'].dt.weekday
train['dropoff_by_weekday'] = train['dropoff_datetime'].dt.weekday

# Creating features based on day

train['pickup_by_day'] = train['pickup_datetime'].dt.day_name()
train['dropoff_by_day'] = train['dropoff_datetime'].dt.day_name()

# Creating features based on Hour

train['pickup_by_hour'] = train['pickup_datetime'].dt.hour
train['dropoff_by_hour'] = train['dropoff_datetime'].dt.hour

## 4. 위치에 기반해서 변수 만들기

****승차 위치와 하차 위치를 통해 움직인 거리를 구한다****
* 하버사인 공식(Haversine Formula) 은 주어진 지점에 대해 구 (Sphere) 의 두 지점 사이의 최단거리(great-circle distance) 를 구하는 공식이다.
* haversine 라이브러리: 두 위경도(Latitude, Longitude) 데이터의 거리를 구할 때 편리함.

$$ 2rarcsin\sqrt{\sin^2(\frac{φ_2 -φ_1}{2}) + cos(φ_1)cos(φ_2)sin^2(\frac{λ_2 -λ_1}{2})} $$

In [ ]:
# haversine 사용 예제

# 위경도 입력
Seoul = (37.541, 126.986)  #Latitude, Longitude
Toronto = (43.65, -79.38)

# 거리 계산
haversine(Seoul, Toronto, unit = 'km')

In [ ]:
# Create a function to determine the distance between two coordinate
def trip_distance(pickup_latitude,pickup_longitude, dropoff_latitude,dropoff_longitude):
    start_place = (pickup_latitude,pickup_longitude)
    finish_place = (dropoff_latitude,dropoff_longitude)
    
    return haversine(start_place,finish_place, unit='km')

In [ ]:
train['distance'] = train.apply(lambda x: trip_distance(x['pickup_latitude'],x['pickup_longitude'],x['dropoff_latitude'],x['dropoff_longitude']), axis=1)

In [ ]:
# Checking to see that distance has been calculated
train['distance'].head()

## 5. target 분포를 확인하고 수정하기

In [ ]:
target = train['trip_duration']

sns.distplot(target, fit = norm)

plt.xlabel('Trip_duration')

이상치로 인해 모양이 이상함 -> boxplot을 그려보기

In [ ]:
sns.boxplot(target)

* 4개의 점이 이상하게 많이 동떨어져 있다.
* 4개만 따로 확인해봐야 될거 같다.

In [ ]:
train[train['trip_duration'] > 1500000]

* 잘못된 데이터일 가능성이 높으므로 삭제하기

In [ ]:
train.drop(train[train['trip_duration'] > 150000].index, axis = 0, inplace = True)

In [ ]:
## 다시 target 분포 확인
sns.distplot(train['trip_duration'], fit = norm)

plt.xlabel('Trip_duration')

In [ ]:
train[train['trip_duration'] >= 20000]

### 승차시간이 너무 크거나 너무 작은 데이터는 삭제해보기
* 전체 승차시간의 대략 25%~75% 이외의 있는 데이터 지우기.

In [ ]:
Q1 = np.percentile(train['trip_duration'], 25) 
Q3 = np.percentile(train['trip_duration'], 75)
IQR = Q3 - Q1 
outlier_step = 1.5 * IQR 
outlier_list_idx = train[train['trip_duration'] > Q3 + outlier_step].index

train.drop(outlier_list_idx, axis = 0, inplace = True)

In [ ]:
sns.distplot(train['trip_duration'], fit = norm)

plt.title('skewness of target {0:.3f}'.format(skew(train['trip_duration'])))